In [10]:
!pip install flask
!pip install pyngrok

In [11]:
!ngrok config add-authtoken 2hguiTCdH1helVkJDvY2c0DywcZ_Vc88G27KizP1sARLP2Gt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [13]:
from flask import Flask, request, render_template_string
from pyngrok import ngrok
from threading import Thread

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    results = None
    error = None
    try:
        if request.method == 'POST':
            state = request.form['state']
            monthly_bill = float(request.form['monthly_bill'])
            roof_area_unit = request.form['roof_area_unit']
            roof_area = float(request.form['roof_area'])
            investment = float(request.form['investment'])
            required_capacity = float(request.form['required_capacity'])
            sanction_load = float(request.form['sanction_load'])
            residential = request.form.get('residential')

            if residential:
                # Assuming some typical values for calculations
                sunlight_hours = 5  # Average sunlight hours per day
                panel_efficiency = 0.19  # 19% efficiency for typical solar panels
                electricity_cost = 6  # Average cost per kWh in Rs
                emission_factor = 0.7  # Tonnes of CO2 saved per MWh of solar energy

                # Convert roof area to square meters if needed
                if roof_area_unit == 'ft2':
                    roof_area = roof_area * 0.092903

                # Calculate solar panel capacity based on roof area and panel efficiency
                panel_capacity_from_area = roof_area * panel_efficiency * sunlight_hours

                # Calculate solar panel capacity based on monthly bill
                monthly_kWh_consumption = monthly_bill / electricity_cost
                panel_capacity_from_bill = monthly_kWh_consumption / (sunlight_hours * 30)

                # Take the lower of the two recommendations
                recommendation = min(panel_capacity_from_area, panel_capacity_from_bill)

                # Calculate yearly energy production
                yearly_energy = recommendation * 365 * sunlight_hours

                # Calculate financial savings
                financial_savings = yearly_energy * electricity_cost

                # Calculate payback period
                payback_period = investment / financial_savings

                # Calculate ROI
                roi = (financial_savings * 25 - investment) / investment * 100

                # Calculate emission savings (in tonnes of CO2)
                emission_savings = yearly_energy / 1000 * emission_factor * 25

                results = {
                    'energy': round(recommendation, 2),
                    'yearly_energy': round(yearly_energy, 2),
                    'financial_savings': round(financial_savings, 2),
                    'payback_period': round(payback_period, 2),
                    'roi': round(roi, 2),
                    'emission_savings': round(emission_savings, 2)
                }
            else:
                error = "Solar calculations are only for residential properties."

        return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Solar Calculator</title>
            <link href="https://cdn.jsdelivr.net/npm/select2@4.1.0-rc.0/dist/css/select2.min.css" rel="stylesheet" />
            <style>
                body { font-family: Arial, sans-serif; margin: 0; padding: 0; display: flex; justify-content: center; align-items: center; height: 100vh; background-color: #f0f0f0; }
                .container { background: white; padding: 20px; border-radius: 8px; box-shadow: 0 0 10px rgba(0, 0, 0, 0.1); width: 80%; max-width: 1200px; margin: auto; }
                .form-group { margin-bottom: 15px; }
                label { display: block; margin-bottom: 5px; }
                input[type="text"], input[type="number"], select { width: 100%; padding: 8px; margin-bottom: 10px; border: 1px solid #ccc; border-radius: 4px; }
                button { padding: 10px 20px; background-color: #007bff; color: white; border: none; border-radius: 4px; cursor: pointer; }
                button:hover { background-color: #0056b3; }
                .row { display: flex; flex-wrap: wrap; }
                .col { flex: 1; padding: 0 15px; }
                .col-half { flex: 0 0 50%; }
                @media (max-width: 768px) {
                    .col-half { flex: 0 0 100%; }
                }
                .error { color: red; }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Solar Rooftop Calculator</h1>
                <form action="/" method="post" id="solarForm">
                    <div class="form-group">
                        <label for="state">Your State:</label>
                        <select id="state" name="state" required>
                            <option value="" disabled selected>Select your state</option>
                            <option value="Andhra Pradesh">Andhra Pradesh</option>
                            <option value="Arunachal Pradesh">Arunachal Pradesh</option>
                            <option value="Assam">Assam</option>
                            <option value="Bihar">Bihar</option>
                            <option value="Chhattisgarh">Chhattisgarh</option>
                            <option value="Goa">Goa</option>
                            <option value="Gujarat">Gujarat</option>
                            <option value="Haryana">Haryana</option>
                            <option value="Himachal Pradesh">Himachal Pradesh</option>
                            <option value="Jharkhand">Jharkhand</option>
                            <option value="Karnataka">Karnataka</option>
                            <option value="Kerala">Kerala</option>
                            <option value="Madhya Pradesh">Madhya Pradesh</option>
                            <option value="Maharashtra">Maharashtra</option>
                            <option value="Manipur">Manipur</option>
                            <option value="Meghalaya">Meghalaya</option>
                            <option value="Mizoram">Mizoram</option>
                            <option value="Nagaland">Nagaland</option>
                            <option value="Odisha">Odisha</option>
                            <option value="Punjab">Punjab</option>
                            <option value="Rajasthan">Rajasthan</option>
                            <option value="Sikkim">Sikkim</option>
                            <option value="Tamil Nadu">Tamil Nadu</option>
                            <option value="Telangana">Telangana</option>
                            <option value="Tripura">Tripura</option>
                            <option value="Uttar Pradesh">Uttar Pradesh</option>
                            <option value="Uttarakhand">Uttarakhand</option>
                            <option value="West Bengal">West Bengal</option>
                        </select>
                    </div>
                    <div class="form-group">
                        <input type="checkbox" id="residential" name="residential">
                        <label for="residential">Residential Property</label>
                    </div>
                    <div class="form-group">
                        <label for="monthly_bill">Average Monthly Bill (Rs):</label>
                        <input type="number" id="monthly_bill" name="monthly_bill" required>
                    </div>
                    <h2>Enter Details for More Accurate Information </h2>
                    <div class="row">
                        <div class="col col-half">
                            <div class="form-group">
                                <label for="roof_area_unit">Total Available Roof Top Area:</label>
                                <input type="radio" id="roof_area_unit" name="roof_area_unit" value="ft2" checked>ft<sup>2</sup>
                                <input type="radio" id="roof_area_unit" name="roof_area_unit" value="m2">m<sup>2</sup>
                            </div>
                        </div>
                        <div class="col col-half">
                            <div class="form-group">
                                <label for="roof_area">Total Roof Top Area:</label>
                                <input type="number" id="roof_area" name="roof_area" required>
                            </div>
                        </div>
                    </div>
                    <div class="row">
                        <div class="col col-half">
                            <div class="form-group">
                                <label for="investment">Amount of Investment (Rs):</label>
                                <input type="number" id="investment" name="investment" required>
                            </div>
                        </div>
                        <div class="col col-half">
                            <div class="form-group">
                                <label for="required_capacity">Required Capacity (kW):</label>
                                <input type="number" id="required_capacity" name="required_capacity" required>
                            </div>
                        </div>
                        <div class="col col-half">
                            <div class="form-group">
                                <label for="sanction_load">Sanction Load (kW):</label>
                                <input type="number" id="sanction_load" name="sanction_load" required>
                            </div>
                        </div>
                    </div>
                    <button type="submit">Calculate</button>
                </form>
                {% if results %}
                    <h2>Results</h2>
                    <ul>
                        <li>Recommended kW for Rooftop Installation: {{ results.energy }} kW</li>
                        <li>Yearly Energy Production: {{ results.yearly_energy }} kWh</li>
                        <li>Annual Financial Savings: Rs {{ results.financial_savings }}</li>
                        <li>Payback Period: {{ results.payback_period }} years</li>
                        <li>Return on Investment (ROI): {{ results.roi }}%</li>
                        <li>25-year Emission Savings: {{ results.emission_savings }} tonnes of CO2</li>
                    </ul>
                {% endif %}
                {% if error %}
                    <p class="error">{{ error }}</p>
                {% endif %}
            </div>
            <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
            <script src="https://cdn.jsdelivr.net/npm/select2@4.1.0-rc.0/dist/js/select2.min.js"></script>
            <script>
                $(document).ready(function() {
                    $('#state').select2();
                });
            </script>
        </body>
        </html>
        ''', results=results, error=error)
    except Exception as e:
        return str(e)

if __name__ == '__main__':
    from pyngrok import ngrok
    from threading import Thread

    # Create a public URL for the Flask app using ngrok
    public_url = ngrok.connect(5000)
    print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

    # Start a new thread to run the Flask app
    thread = Thread(target=app.run, kwargs={"use_reloader": False})
    thread.start()


 * ngrok tunnel "NgrokTunnel: "https://8a0e-104-196-61-39.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
